Cigarette Smoking detection using YOLOv3 darknet
---

In [1]:
!apt update
!apt upgrade -y
!uname -m && cat /etc/*release
!gcc --version
!uname -r

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,867

In [2]:
ls

sample_data/


To access Google Drive Folder and Files

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls -a "/content/drive/My Drive/KISTI_RECYCLE/DATA/"

 캔	    종이상자
 유리	    플라스틱
 비닐	    darknet53.conv.74
 종이류    'darknet_yolov3_detection_ipynb의_사본 (2).py'
 페트병     Darknet_YOLOv3_Detection_ipynb의_사본.ipynb
 스프레이   images
 종이팩     plasticbag
 컵라면     plasticbagxml
 스티로폼   ssd_mobilenet_v2_coco_2018_03_29
 아이스팩   training


**Original Version of Darknet**

In [5]:
%rm -r darknet
!git clone https://github.com/pjreddie/darknet
%cd darknet/

rm: cannot remove 'darknet': No such file or directory
Cloning into 'darknet'...
remote: Enumerating objects: 5913, done.
remote: Total 5913 (delta 0), reused 0 (delta 0), pack-reused 5913
Receiving objects: 100% (5913/5913), 6.34 MiB | 30.05 MiB/s, done.
Resolving deltas: 100% (3921/3921), done.
/content/darknet


**Modify Version of Darknet**

In [6]:
'''#%rm -r darknet
!git clone https://github.com/AlexeyAB/darknet/
%cd darknet/'''

Cloning into 'darknet'...
remote: Enumerating objects: 14291, done.
remote: Total 14291 (delta 0), reused 0 (delta 0), pack-reused 14291
Receiving objects: 100% (14291/14291), 12.84 MiB | 18.16 MiB/s, done.
Resolving deltas: 100% (9748/9748), done.
/content/darknet/darknet


In [6]:
!apt install libopencv-dev python-opencv ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
libopencv-dev is already the newest version (3.2.0+dfsg-4ubuntu0.1).
The following NEW packages will be installed:
  python-opencv
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 535 kB of archives.
After this operation, 2,944 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python-opencv amd64 3.2.0+dfsg-4ubuntu0.1 [535 kB]
Fetched 535 kB in 1s (449 kB/s)
Selecting previously unselected package python-opencv.
(Reading database ... 144599 files and directories currently installed.)
Preparing to unpack .../python-opencv_3.2.0+dfsg-4ubuntu0.1_amd64.deb ...
Unpacking python-opencv (3.2.0+dfsg-4ubuntu0.1) ...
Setting up python-opencv (3.2.0+dfsg-4ubuntu0.1) ...


Changing the variables to include OpenCV and GPU in the Makefile

In [7]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
#!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
%pycat Makefile

In [8]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:957:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result 

**How to train (to detect your custom objects):**
---

**Training Yolo v3:**

1.Create file yolo-obj.cfg with the same content as in yolov3.cfg (or copy yolov3.cfg to yolo-obj.cfg) and:

* change line batch to batch=64
* change line subdivisions to subdivisions=8
* change line classes=80 to your number of objects in each of 3 [yolo]-layers:
  * yolov3.cfg#L610
  * yolov3.cfg#L696
  * yolov3.cfg#L783
* change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer
  * yolov3.cfg#L603
  * yolov3.cfg#L689
  * yolov3.cfg#L776
  
So if classes=1 then should be filters=18. If classes=2 then write filters=21.

In [9]:
%cp cfg/yolov3.cfg cfg/yolo-obj.cfg
!sed -i 's/batch=1/batch=64/g' cfg/yolo-obj.cfg
!sed -i 's/subdivisions=1/subdivisions=8/g' cfg/yolo-obj.cfg
!sed -i 's/classes=80/classes=1/g' cfg/yolo-obj.cfg
!sed -i 's/filters=255/filters=18/g' cfg/yolo-obj.cfg
!sed -i 's/width=416/width=320/g' cfg/yolo-obj.cfg
!sed -i 's/height=416/height=320/g' cfg/yolo-obj.cfg

In [10]:
%pycat cfg/yolo-obj.cfg

2.Create file obj.names in the directory `build\darknet\x64\data\` with objects names - each in new line



In [11]:
all_classes = """plasticbag
"""

file = """text_file = open("/content/darknet/data/obj.names", "w");text_file.write(all_classes);text_file.close()""" 

exec(file)
%pycat /content/darknet/data/obj.names

3.Create file obj.data in the directory `build\darknet\x64\data\` containing (where classes = number of objects):

In [12]:
obj_data = """classes= 1
train  = /content/darknet/data/train.txt
valid  = /content/darknet/data/valid.txt
names = /content/darknet/data/obj.names
backup = /content/darknet/backup/
"""

file = """text_file = open("/content/darknet/data/obj.data", "w");text_file.write(obj_data);text_file.close()""" 

exec(file)
%pycat /content/darknet/data/obj.data

In [13]:
%cd "/content/drive/My Drive/KISTI_RECYCLE/DATA/plasticbag"

/content/drive/My Drive/KISTI_RECYCLE/DATA/plasticbag


4.Put image-files (.jpg) of your objects in the directory build/darknet/x64/data/obj/


In [14]:
#%mkdir build/darknet/x64/data/obj
%cp -r "/content/drive/My Drive/KISTI_RECYCLE/DATA/plasticbag" /content/darknet/data/obj/

In [15]:
%ls -1 /content/darknet/data/obj/*.jpg | wc -l
%ls -1 /content/darknet/data/obj/*.txt | wc -l

101
101


5.You should label each object on images from your dataset. Use this visual GUI-software for marking bounded boxes of objects and generating annotation files for Yolo v2 & v3: 

---
**LabelImg**
  
LabelImg is a graphical image annotation tool.: 
https://github.com/tzutalin/labelImg


---

**Yolo_mark**

Windows & Linux GUI for marking bounded boxes of objects in images for training Yolo v3 and v2

https://github.com/AlexeyAB/Yolo_mark

It will create `.txt`-file for each `.jpg`-image-file - in the same directory and with the same name, but with `.txt`-extension, and put to file: object number and object coordinates on this image, for each object in new line: `<object-class> <x> <y> <width> <height>`

Where:


*   `<object-class>` - integer object number from 0 to (classes-1)
*   `<x_center> <y_center> <width> <height>` - float values relative to width and height of image, it can be equal from (0.0 to 1.0]
*   for example: `<x> = <absolute_x> / <image_width>` or `<height> = <absolute_height> / <image_height>`
*   atention: `<x_center> <y_center>` - are center of rectangle (are not top-left corner)


For example for img1.jpg you will be created img1.txt containing:


```
1 0.716797 0.395833 0.216406 0.147222
0 0.687109 0.379167 0.255469 0.158333
1 0.420312 0.395833 0.140625 0.166667
```



6.Create file train.txt and valid.txt in directory `build\darknet\x64\data\` with filenames of your images, each filename in new line, with path relative to darknet, for example containing:


In [16]:
import os, fnmatch
import numpy as np

train_file = open("/content/darknet/data/train.txt", "w")
valid_file = open("/content/darknet/data/valid.txt", "w")
listOfFiles = os.listdir('/content/darknet/data/obj/')  
pattern = "*.jpg"  
for f_name in listOfFiles:  
  if fnmatch.fnmatch(f_name, pattern):
    if np.random.rand(1) < 0.8:
      train_file.write("/content/darknet/data/obj/"+f_name+"\n")
      #print ("data/obj/"+f_name)
    else:
      valid_file.write("/content/darknet/data/obj/"+f_name+"\n")  
      
train_file.close()
valid_file.close()

In [17]:
#Count number of files 
!wc -l /content/darknet/data/train.txt
!wc -l /content/darknet/data/valid.txt

73 /content/darknet/data/train.txt
28 /content/darknet/data/valid.txt


In [18]:
%pycat /content/darknet/data/valid.txt

7.Download pre-trained weights for the convolutional layers (154 MB): https://pjreddie.com/media/files/darknet53.conv.74 and put to the directory `build\darknet\x64`

In [21]:
!wget -P /content/darknet/ https://pjreddie.com/media/files/darknet53.conv.74
#%ls /content/darknet/build/darknet/x64/

--2020-09-16 19:10:16--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘/content/darknet/darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   140KB/s    in 18m 36s 

2020-09-16 19:28:54 (142 KB/s) - ‘/content/darknet/darknet53.conv.74’ saved [162482580/162482580]



In [19]:
%cp -r "/content/drive/My Drive/KISTI_RECYCLE/darknet53.conv.74" /content/darknet/

8.training by using the command line:


```
./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/darknet53.conv.74 -dont_show
```

8.1. For training with mAP (mean average precisions) calculation for each 4 Epochs (set valid=valid.txt or train.txt in obj.data file) and run: 



```
./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/darknet53.conv.74 -dont_show -map
```



In [30]:
! /content/darknet/darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolo-obj.cfg /content/darknet/darknet53.conv.74 -dont_show

Region 82 Avg IOU: 0.289615, Class: 0.666371, Obj: 0.013331, No Obj: 0.007303, .5R: 0.125000, .75R: 0.000000,  count: 8
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.003153, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.001428, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.344130, Class: 0.587894, Obj: 0.017170, No Obj: 0.007140, .5R: 0.375000, .75R: 0.000000,  count: 8
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.003154, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.001416, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.405686, Class: 0.612385, Obj: 0.024549, No Obj: 0.007343, .5R: 0.125000, .75R: 0.000000,  count: 8
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.003107, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.001452, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.339412, Class: 0

9.After training is complete - get result yolo-obj_final.weights from path `build\darknet\x64\backup\`


*   After each 100 iterations you can stop and later start training from this point. For example, after 2000 iterations you can stop training, and later just copy yolo-obj_2000.weights from `build\darknet\x64\backup\` to` build\darknet\x64\ `and start training using: 


```
./darknet detector train data/obj.data yolo-obj.cfg yolo-obj_2000.weights
```





In [32]:
!/content/darknet/darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolo-obj.cfg /content/darknet/backup/yolo-obj_200.weights -dont_show

yolo-obj
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   320 x 320 x   3   ->   320 x 320 x  32  0.177 BFLOPs
    1 conv     64  3 x 3 / 2   320 x 320 x  32   ->   160 x 160 x  64  0.944 BFLOPs
    2 conv     32  1 x 1 / 1   160 x 160 x  64   ->   160 x 160 x  32  0.105 BFLOPs
    3 conv     64  3 x 3 / 1   160 x 160 x  32   ->   160 x 160 x  64  0.944 BFLOPs
    4 res    1                 160 x 160 x  64   ->   160 x 160 x  64
    5 conv    128  3 x 3 / 2   160 x 160 x  64   ->    80 x  80 x 128  0.944 BFLOPs
    6 conv     64  1 x 1 / 1    80 x  80 x 128   ->    80 x  80 x  64  0.105 BFLOPs
    7 conv    128  3 x 3 / 1    80 x  80 x  64   ->    80 x  80 x 128  0.944 BFLOPs
    8 res    5                  80 x  80 x 128   ->    80 x  80 x 128
    9 conv     64  1 x 1 / 1    80 x  80 x 128   ->    80 x  80 x  64  0.105 BFLOPs
   10 conv    128  3 x 3 / 1    80 x  80 x  64   ->    80 x  80 x 128  0.944 BFLOPs
   11 res    8              

Custom object detection:
---
Example of Cigarette Smoking detection:

**Image : **

```
./darknet detector test build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/smoking_1000it.weights -thresh 0.20 data/cigarette.jpg
```


**Video : **

```
./darknet detector demo build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/smoking_1000it.weights -thresh 0.20 -dont_show Smoking.mp4 -out_filename Smoking_output.mp4
```



In [24]:
! /content/darknet/darknet detector demo /content/darknet/data/obj.data /content/darknet/cfg/yolo-obj.cfg /content/darknet/backup/yolo-obj_200.weights

Cannot load image "data/labels/32_0.png"
Cannot load image "data/labels/33_0.png"
Cannot load image "data/labels/34_0.png"
Cannot load image "data/labels/35_0.png"
Cannot load image "data/labels/36_0.png"
Cannot load image "data/labels/37_0.png"
Cannot load image "data/labels/38_0.png"
Cannot load image "data/labels/39_0.png"
Cannot load image "data/labels/40_0.png"
Cannot load image "data/labels/41_0.png"
Cannot load image "data/labels/42_0.png"
Cannot load image "data/labels/43_0.png"
Cannot load image "data/labels/44_0.png"
Cannot load image "data/labels/45_0.png"
Cannot load image "data/labels/46_0.png"
Cannot load image "data/labels/47_0.png"
Cannot load image "data/labels/48_0.png"
Cannot load image "data/labels/49_0.png"
Cannot load image "data/labels/50_0.png"
Cannot load image "data/labels/51_0.png"
Cannot load image "data/labels/52_0.png"
Cannot load image "data/labels/53_0.png"
Cannot load image "data/labels/54_0.png"
Cannot load image "data/labels/55_0.png"
Cannot load imag

In [49]:
import IPython
js_code = '''
document.querySelector("#output-area").appendChild(document.createTextNode("hello world!"));
'''
display(IPython.display.Javascript(js_code))

<IPython.core.display.Javascript object>

In [21]:
%cp -r "/content/drive/My Drive/KISTI_RECYCLE/yolo-obj_200.weights" /content/darknet/backup/

In [23]:
!/content/darknet/darknet detector test /content/darknet/data/obj.data /content/darknet/cfg/yolo-obj.cfg /content/darknet/backup/yolo-obj_200.weights -thresh 0.20 /content/darknet/data/plasticbag.jpg

Cannot load image "data/labels/32_0.png"
Cannot load image "data/labels/33_0.png"
Cannot load image "data/labels/34_0.png"
Cannot load image "data/labels/35_0.png"
Cannot load image "data/labels/36_0.png"
Cannot load image "data/labels/37_0.png"
Cannot load image "data/labels/38_0.png"
Cannot load image "data/labels/39_0.png"
Cannot load image "data/labels/40_0.png"
Cannot load image "data/labels/41_0.png"
Cannot load image "data/labels/42_0.png"
Cannot load image "data/labels/43_0.png"
Cannot load image "data/labels/44_0.png"
Cannot load image "data/labels/45_0.png"
Cannot load image "data/labels/46_0.png"
Cannot load image "data/labels/47_0.png"
Cannot load image "data/labels/48_0.png"
Cannot load image "data/labels/49_0.png"
Cannot load image "data/labels/50_0.png"
Cannot load image "data/labels/51_0.png"
Cannot load image "data/labels/52_0.png"
Cannot load image "data/labels/53_0.png"
Cannot load image "data/labels/54_0.png"
Cannot load image "data/labels/55_0.png"
Cannot load imag

In [46]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [47]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

'NoneType' object has no attribute 'split'


In [ ]:
ls -lh *.mp4

**Playing videos on google colab**

In [ ]:
import io
import base64
from IPython.display import HTML
video = io.open('Smoking_20%.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''.format(encoded.decode('ascii')))

Extra Tutorial
---

Copying data from google colab to google drive

In [37]:
%cp -r /content/darknet/backup/yolo-obj_200.weights "/content/drive/My Drive/KISTI_RECYCLE/"

Downloading the data from the colab

In [38]:
from google.colab import files
files.download('/content/darknet/backup/yolo-obj_200.weights')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Copying data from google drive to google colab

In [ ]:
%cp -r "/content/drive/My Drive/KISTI_RECYCLE/yolo-obj_200.weights" build/darknet/
#%cp -r "/content/drive/My Drive/GColab/SmokingDEMO.mp4" .

Youtube Link: 
---
https://youtu.be/vEnQIptZzyI